# Basic Usage: Loading and Exploring Braidz Data

This notebook demonstrates the fundamental workflow for loading and exploring
trajectory data from `.braidz` files.

## What you'll learn

1. Loading single and multiple braidz files
2. Exploring the data structure
3. Basic trajectory filtering
4. Computing and visualizing kinematics

In [ ]:
import braidz_analysis as ba
import matplotlib.pyplot as plt
import numpy as np

# Print version
print(f"braidz_analysis version: {ba.__version__}")

## 1. Loading Data

The `read_braidz()` function loads trajectory data from `.braidz` files.

**Note:** Update the `DATA_PATH` and `BRAIDZ_FILE` variables to point to your data.

In [ ]:
# === CONFIGURE YOUR DATA PATH HERE ===
DATA_PATH = "/path/to/your/experiments"  # Update this!
BRAIDZ_FILE = "your_experiment.braidz"   # Update this!

# Load a single file
data = ba.read_braidz(BRAIDZ_FILE, base_folder=DATA_PATH)

# Display summary
print(data)

### Loading Multiple Files

You can load multiple files at once. They will be combined with an `exp_num` column to distinguish them.

In [ ]:
# Load multiple files (example - update with your file names)
# files = ["exp1.braidz", "exp2.braidz", "exp3.braidz"]
# data = ba.read_braidz(files, base_folder=DATA_PATH, progressbar=True)

## 2. Exploring the Data Structure

The `BraidzData` object contains:
- `trajectories`: Main DataFrame with position and velocity data
- `opto`: Optogenetic event data (if present)
- `stim`: Visual stimulus event data (if present)

In [ ]:
# Examine the trajectory data
print("Trajectory columns:")
print(data.trajectories.columns.tolist())
print(f"\nShape: {data.trajectories.shape}")
print(f"Unique trajectories: {data.trajectories['obj_id'].nunique()}")

In [ ]:
# Preview the data
data.trajectories.head()

In [ ]:
# Check for opto/stim data
print(f"Has opto data: {data.has_opto}")
print(f"Has stim data: {data.has_stim}")

if data.has_opto:
    print(f"\nOpto events: {len(data.opto)}")
    print(f"Opto columns: {data.opto.columns.tolist()}")

## 3. Trajectory Filtering

Filter trajectories based on quality criteria (length, position, movement).

In [ ]:
# Filter trajectories with default criteria
filtered_df = ba.filter_trajectories(data.trajectories)

print(f"Original trajectories: {data.trajectories['obj_id'].nunique()}")
print(f"After filtering: {filtered_df['obj_id'].nunique()}")

In [ ]:
# Custom filtering with Config
strict_config = ba.Config(
    min_trajectory_frames=300,  # Longer trajectories only
    z_bounds=(0.1, 0.25),        # Tighter z range
    max_radius=0.2,              # Closer to center
)

strict_filtered = ba.filter_trajectories(data.trajectories, config=strict_config)
print(f"With strict filters: {strict_filtered['obj_id'].nunique()}")

## 4. Computing Kinematics

Add kinematic columns (velocity, heading, flight state) to a trajectory.

In [ ]:
# Get a single trajectory for demonstration
obj_ids = filtered_df['obj_id'].unique()
if len(obj_ids) > 0:
    example_traj = filtered_df[filtered_df['obj_id'] == obj_ids[0]].copy()
    
    # Add kinematics
    example_traj = ba.add_kinematics_to_trajectory(example_traj)
    
    print("New columns added:")
    print([c for c in example_traj.columns if c not in data.trajectories.columns])

In [ ]:
# Plot kinematics for one trajectory
if len(obj_ids) > 0:
    fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
    
    time_s = (example_traj['frame'] - example_traj['frame'].iloc[0]) / 100
    
    # Angular velocity
    axes[0].plot(time_s, np.degrees(example_traj['angular_velocity']))
    axes[0].set_ylabel('Angular Velocity (deg/s)')
    axes[0].axhline(0, color='gray', linestyle='--', alpha=0.5)
    
    # Linear velocity
    axes[1].plot(time_s, example_traj['linear_velocity'])
    axes[1].set_ylabel('Linear Velocity (m/s)')
    
    # Flight state
    axes[2].fill_between(time_s, 0, example_traj['is_flying'].astype(int), alpha=0.3)
    axes[2].set_ylabel('Flying')
    axes[2].set_xlabel('Time (s)')
    axes[2].set_ylim(-0.1, 1.1)
    
    plt.tight_layout()
    plt.show()

## 5. Visualizing Trajectories

Plot the spatial path of trajectories.

In [ ]:
# Plot multiple trajectories in 2D
fig, ax = plt.subplots(figsize=(8, 8))

for obj_id in obj_ids[:10]:  # Plot first 10 trajectories
    traj = filtered_df[filtered_df['obj_id'] == obj_id]
    ax.plot(traj['x'], traj['y'], alpha=0.5, linewidth=0.5)

# Add arena boundary (if applicable)
theta = np.linspace(0, 2*np.pi, 100)
ax.plot(0.23 * np.cos(theta), 0.23 * np.sin(theta), 'k--', alpha=0.3, label='Arena')

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_aspect('equal')
ax.legend()
ax.set_title('Fly Trajectories (Top View)')
plt.show()

## Summary

In this notebook, you learned how to:

1. Load braidz files using `ba.read_braidz()`
2. Explore the `BraidzData` structure
3. Filter trajectories using `ba.filter_trajectories()`
4. Compute kinematics using `ba.add_kinematics_to_trajectory()`
5. Visualize trajectories

**Next steps:**
- See `02_opto_analysis.ipynb` for optogenetic response analysis
- See `03_saccade_analysis.ipynb` for saccade detection and characterization